In [1]:
import sys
print(sys.version)
print(sys.executable)


3.12.12 (main, Oct  9 2025, 11:07:00) [Clang 17.0.0 (clang-1700.0.13.3)]
/Users/nami/Desktop/python scripts/spark-env/bin/python


In [2]:
import findspark, os
os.environ["JAVA_HOME"] = "/opt/homebrew/Cellar/openjdk@11/11.0.29/libexec/openjdk.jdk/Contents/Home"

findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("crunchbase_odm_orgs").getOrCreate()
spark


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 11:24:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.conf.set("spark.sql.csv.parser.columnPruning.enabled", "true")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")  #safer date parsing


In [4]:
#read the Crunchbase ODM Orgs CSV into a Spark DataFrame
file_path = "/Users/nami/Desktop/python scripts/crunchbase_odm_orgs.csv"

cb_sdf = spark.read.csv(
    file_path,
    header=True,          #first row are column names
    inferSchema=True,     #let Spark infer types
    multiLine=True,       #safer if any embedded newlines in quoted text
    escape="\\",          #common escape setting
    mode="PERMISSIVE",    #don’t fail on the first bad row
)

cb_sdf.printSchema()
cb_sdf.show(5, truncate=False)

root
 |-- uuid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- primary_role: string (nullable = true)
 |-- cb_url: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- homepage_url: string (nullable = true)
 |-- logo_url: string (nullable = true)
 |-- facebook_url: string (nullable = true)
 |-- twitter_url: string (nullable = true)
 |-- linkedin_url: string (nullable = true)
 |-- combined_stock_symbols: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- short_description: string (nullable = true)

+------------------------------------+--------+------------+------------+-------------------------------------------------------------------------------------------------------------+------------+------------------------+-------------------------------------------------------------------------------------------------------------

In [5]:
#print the count of all records
total_records = cb_sdf.count()
print("Total records:", total_records)


Total records: 1127636


In [7]:
#find all companies with names that contain exactly three words using PySpark DataFrame
#split the company name into words, count them, and filter for exactly 3
cb_three_word_name = cb_sdf.where(
    (F.col("name").isNotNull()) &  # skip nulls
    (F.size(F.split(F.col("name"), r"\s+")) == 3)
)

#count such companies
three_word_count = cb_three_word_name.count()
print("Number of companies with exactly three-word names:", three_word_count)

#display selected columns using show()
cb_three_word_name.select("name", "city", "region", "country_code").show(truncate=False)


Number of companies with exactly three-word names: 206296
+-----------------------------+-------------+----------------+------------+
|name                         |city         |region          |country_code|
+-----------------------------+-------------+----------------+------------+
|Fox Interactive Media        |Beverly Hills|California      |USA         |
|Hutchison Whampoa Limited    |Hong Kong    |Hong Kong Island|HKG         |
|Gannett Co Inc               |Mclean       |Virginia        |USA         |
|HOT or NOT                   |San Francisco|California      |USA         |
|Funny Or Die                 |San Mateo    |California      |USA         |
|Peak Steal Buildings         |Morgan       |Georgia         |USA         |
|Kosiso - store               |Chicago      |Illinois        |USA         |
|The Accelerator Group        |Mclean       |Virginia        |USA         |
|Real Time Matrix             |Oakland      |California      |USA         |
|Keep Highways Safe           

In [ ]:
#find all companies located in the state of New Jersey with Pyspark DataFrame
#filter for companies whose region is exactly "New Jersey"
cb_nj = cb_sdf.filter(F.col("region") == "New Jersey")

#count how many companies satisfy this condition
nj_count = cb_nj.count()
print("Number of companies located in New Jersey:", nj_count)

#show only selected columns (name + location fields)
cb_nj.select("name", "city", "region", "country_code").show(truncate=False)



Number of companies located in New Jersey: 10251
+-------------------+--------------+----------+------------+
|name               |city          |region    |country_code|
+-------------------+--------------+----------+------------+
|Blogsigs           |Madison       |New Jersey|USA         |
|MSNBC              |Secaucus      |New Jersey|USA         |
|BitWine            |Tenafly       |New Jersey|USA         |
|GoAmerica          |Hackensack    |New Jersey|USA         |
|Phone.com          |Newark        |New Jersey|USA         |
|Phanfare           |Princeton     |New Jersey|USA         |
|Vidyo              |Hackensack    |New Jersey|USA         |
|eventsbot          |Piscataway    |New Jersey|USA         |
|CheerOutLoud       |Perth Amboy   |New Jersey|USA         |
|Jet Numbers        |East Brunswick|New Jersey|USA         |
|ShareMethods       |South Orange  |New Jersey|USA         |
|Winescorecard      |Lambertville  |New Jersey|USA         |
|ePrep              |Princeton     |

In [9]:
#add a new column TechDomain: 1 if domain contains "github.io", else 0 with PySpark DataFrame
cb_with_techdomain = cb_sdf.withColumn(
    "TechDomain",
    F.when(F.col("domain").contains("github.io"), 1).otherwise(0)
)

#count how many companies fall in this category (optional)
techdomain_count = cb_with_techdomain.filter(F.col("TechDomain") == 1).count()
print("Number of companies with github.io in their domain:", techdomain_count)

#display only name, location, and TechDomain columns where TechDomain = 1
cb_with_techdomain.filter(F.col("TechDomain") == 1) \
    .select("name", "city", "region", "country_code", "TechDomain") \
    .show(truncate=False)


Number of companies with github.io in their domain: 81


+----------------------------------------+-------------+------------+------------+----------+
|name                                    |city         |region      |country_code|TechDomain|
+----------------------------------------+-------------+------------+------------+----------+
|OSSEC                                   |San Francisco|California  |USA         |1         |
|Fosstrak                                |San Francisco|California  |USA         |1         |
|littlesnapper                           |NULL         |NULL        |NULL        |1         |
|Associação CRIArte                      |NULL         |NULL        |NULL        |1         |
|Bros Labs                               |Hanoi        |NA - Vietnam|VNM         |1         |
|Telcial                                 |Brenham      |Texas       |USA         |1         |
|Dev Free Casts                          |NULL         |NULL        |NULL        |1         |
|Ojas Open Platform                      |San Francisco|Cali

In [ ]:
#Find all companies whose names contain at least one repeating consecutive letter using Spark UDF
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf

#define a normal Python function for the custom logic
def has_repeating_consecutive_letter(name: str) -> bool:
    if not name:
        return False
    s = name.strip()
    for i in range(len(s) - 1):
        # check if two consecutive letters are the same (case-insensitive)
        if s[i].isalpha() and s[i+1].isalpha() and s[i].lower() == s[i+1].lower():
            return True
    return False

#register it as a Spark UDF
has_repeat_udf = udf(has_repeating_consecutive_letter, BooleanType())

#apply the UDF as a filter on the DataFrame
cb_repeating = cb_sdf.filter(has_repeat_udf(F.col("name")))

#count the matching companies
repeat_count = cb_repeating.count()
print("Number of companies with repeating consecutive letters:", repeat_count)

#display only relevant columns
cb_repeating.select("name", "city", "region", "country_code").show(truncate=False)


Number of companies with repeating consecutive letters: 273416
+---------------------+----------------+-----------+------------+
|name                 |city            |region     |country_code|
+---------------------+----------------+-----------+------------+
|Digg                 |New York        |New York   |USA         |
|Facebook             |Menlo Park      |California |USA         |
|Accel                |Palo Alto       |California |USA         |
|Twitter              |San Francisco   |California |USA         |
|MeetMoi              |New York        |New York   |USA         |
|Joost                |New York        |New York   |USA         |
|AddThis              |Vienna          |Virginia   |USA         |
|Thoof                |Austin          |Texas      |USA         |
|Hearst Communications|New York        |New York   |USA         |
|Gannett Co Inc       |Mclean          |Virginia   |USA         |
|AllPeers             |Oxford          |Oxfordshire|GBR         |
|Aggregate Kn

25/11/08 13:31:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 306019 ms exceeds timeout 120000 ms
25/11/08 13:31:30 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/08 13:31:32 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$